## Load up data and generate features

In [1]:
import sys
sys.path.append('..')

import audio
import features
import glob
import numpy as np

In [47]:
file_set = 'train'
for cat in ( "boat","heli",):
    ctr = 0
    #files = glob.glob("./downloads_" + cat + "/processed/*.wav")
    file_path = "../../data/" +  cat + '/' + file_set + "/*.wav"
    files = glob.glob(file_path)
    obs_count = 0
    # Generate features
    for file in files[:100]:
        ctr += 1
#         if np.random.uniform(0,1,1)[0] < 0.8:
#             print("skipping",ctr,file)
#             continue
        print("Loading " + file,ctr)
        raw = audio.import_wav(file)

        # Generate each type of feature
        corr = features.gen_correlogram(raw)
        mag_spec = features.gen_mag_spec(raw)
        #constQ = features.gen_constantQ(raw)
#         cnn = features.gen_cnn(raw, use_gpu=False)

        # Make sure all have correct format
        corr = corr.T 
        
        Y = 1 if cat == 'boat' else 0

        # Export each feature separately for various types of comparison
        with open('../../data/' + file_set +'/features_corr.csv', 'ab') as f_handle:
            np.savetxt(f_handle, np.concatenate((corr, np.ones((corr.shape[0],1)) * Y ),axis=1)  , fmt='%.6e', delimiter=',')
            
        #with open('../../data/' + file_set +'/features_mag_spec.csv', 'ab') as f_handle:
        #    np.savetxt(f_handle, np.concatenate((mag_spec, np.ones((mag_spec.shape[0],1)) * Y ),axis=1), fmt='%.6e', delimiter=',')
            
        with open('../../data/' + file_set + '/features_constQ.csv', 'ab') as f_handle:
            np.savetxt(f_handle, np.concatenate((constQ, np.ones((constQ.shape[0],1)) * Y),axis=1) , fmt='%.6e', delimiter=',')
            
#         with open('../../data/' + cat + '/features_cnn.csv', 'ab') as f_handle:
#             np.savetxt(f_handle, np.concatenate(cnn, np.ones(corr.shape[0],1) * Y ) , fmt='%.6e', delimiter=',')

    print("Nice!")

Loading ../../data/boat/train\03LBxCR55J4_000.wav 1
Loading ../../data/boat/train\03LBxCR55J4_001.wav 2
Loading ../../data/boat/train\03LBxCR55J4_002.wav 3
Loading ../../data/boat/train\03LBxCR55J4_003.wav 4
Loading ../../data/boat/train\03LBxCR55J4_004.wav 5
Loading ../../data/boat/train\03LBxCR55J4_005.wav 6
Loading ../../data/boat/train\03LBxCR55J4_006.wav 7
Loading ../../data/boat/train\03LBxCR55J4_007.wav 8
Loading ../../data/boat/train\03LBxCR55J4_008.wav 9
Loading ../../data/boat/train\03LBxCR55J4_009.wav 10
Loading ../../data/boat/train\03LBxCR55J4_010.wav 11
Loading ../../data/boat/train\03LBxCR55J4_011.wav 12
Loading ../../data/boat/train\03LBxCR55J4_012.wav 13
Loading ../../data/boat/train\03LBxCR55J4_013.wav 14
Loading ../../data/boat/train\03LBxCR55J4_014.wav 15
Loading ../../data/boat/train\03LBxCR55J4_015.wav 16
Loading ../../data/boat/train\03LBxCR55J4_016.wav 17
Loading ../../data/boat/train\03LBxCR55J4_017.wav 18
Loading ../../data/boat/train\03LBxCR55J4_018.wav 19
Lo

In [34]:
file_path

'../../data/train/heli/*.wav'

## Train model

In [ ]:
train_fcorr = np.genfromtxt('CNN_features/train/features_corr.csv',delimiter=',')
print(train_fcorr.shape)
train_cq = np.genfromtxt('CNN_features/train/features_constQ.csv',delimiter=',')
print(train_cq.shape)
# train_mag_spec = np.genfromtxt('../../data/train/features_mag_spec.csv',delimiter=',')
# print(train_mag_spec.shape)
print("done")
test_fcorr = np.genfromtxt('CNN_features/data/test/features_corr.csv',delimiter=',')
print(test_fcorr.shape)
test_cq = np.genfromtxt('../../data/test/features_constQ.csv',delimiter=',')
print(test_cq.shape)
# test_mag_spec = np.genfromtxt('../../data/test/features_mag_spec.csv',delimiter=',')
# print(test_mag_spec.shape)
print("done")

In [39]:
trainY = train_fcorr[:,-1]
trainX = train_fcorr[:,:-1]

testY = test_fcorr[:,-1]
testX = test_fcorr[:,:-1]

In [40]:
trainX.shape, testX.shape, trainY.shape, testY.shape

((104487, 1280), (43165, 1280), (104487,), (43165,))

In [11]:
sum(trainY)

11213.0

In [44]:
from sklearn.mixture import GaussianMixture as GMM

train_boat =  trainX[trainY==1]  #X_boat[:-5120]
train_heli = trainX[trainY==0]
test_boat = testX[testY==1]
test_heli = testX[testY==0]

gmm = {'boat':None, 'heli':None}

gmm["boat"] = GMM(n_components=2)
gmm["boat"].fit(train_boat)

gmm["heli"] = GMM(n_components=2)
gmm["heli"].fit(train_heli)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
        means_init=None, n_components=2, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

## Predict

In [45]:
scores = np.zeros((testY.shape[0], 2))

scores[:, 0] = gmm["boat"].score_samples(testX)
scores[:, 1] = gmm["heli"].score_samples(testX)

pred = np.greater(scores[:, 1], scores[:, 0])
error = np.not_equal(pred, np.squeeze(testY))

error_rate = np.sum(error) / len(error)
print(error_rate)

0.428796478629


In [41]:
from sklearn.linear_model import LogisticRegression

In [42]:
lr = LogisticRegression(penalty='l1')
lr.fit(trainX, trainY)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [43]:
print(lr.score(trainX, trainY))
lr.score(testX, testY)

0.756180194665


0.45601760685740761

In [33]:
import numpy as np
train_heli = np.genfromtxt('CNN_features/train/heli/features_cnn.csv',delimiter=',')
heli_labels = np.ones((train_heli.shape[0], 1))
train_heli = np.hstack((train_heli, heli_labels))

In [34]:
train_boat = np.genfromtxt('CNN_features/train/boat/features_cnn.csv',delimiter=',')
boat_labels = np.zeros((train_boat.shape[0], 1))
train_boat = np.hstack((train_boat, boat_labels))

In [35]:
total_train = np.vstack((train_heli, train_boat))

In [39]:
test_heli = np.genfromtxt('CNN_features/test/heli/features_cnn.csv',delimiter=',')
heli_labels = np.ones((test_heli.shape[0], 1))
test_heli = np.hstack((test_heli, heli_labels))

In [40]:
test_boat = np.genfromtxt('CNN_features/test/boat/features_cnn.csv',delimiter=',')
boat_labels = np.zeros((test_boat.shape[0], 1))
test_boat = np.hstack((test_boat, boat_labels))

In [41]:
total_test = np.vstack((test_heli, test_boat))

In [64]:
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
pca = PCA(n_components=15, svd_solver='arpack')
lr = LogisticRegression(penalty='l1')
trainX = total_train[:, :-1]
trainY = total_train[:, -1]
testX = total_test[:, :-1]
testY = total_test[:, -1]
trainX = pca.fit_transform(trainX)
testX = pca.transform(testX)
# trainY = trainY.reshape(trainY.shape[0], -1)
trainX.shape, trainY.shape

((617, 15), (617,))

In [65]:
lr.fit(trainX, trainY)
print(lr.score(trainX, trainY))
lr.score(testX, testY)

0.90113452188


0.66382978723404251

In [42]:
total_test[:, -1]

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0